**Reference** https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-register-datasets

In [1]:
import numpy as np


def create_data(no_samples=100,dx=256,dy=256,dz=1):    
    #create training data.
    X=np.random.randn(no_samples,dx,dy,dz) #0 mean 1 std_dev
    #generate reference data (label/true data)
    Y=0.8*X+2   #0.8 std_dev 2 mean
    print(X.shape,Y.shape)
    return (X,Y)    





In [2]:
X_train,y_train=create_data(no_samples=50000)
X_val,y_val=create_data(no_samples=10000)
X_test,y_test=create_data(no_samples=20000)

(50000, 256, 256, 1) (50000, 256, 256, 1)
(10000, 256, 256, 1) (10000, 256, 256, 1)
(20000, 256, 256, 1) (20000, 256, 256, 1)


For Storing large numpy arrays I am using a memmap https://docs.scipy.org/doc/numpy/reference/generated/numpy.memmap.html

In [11]:
#save the numpy files to disk on the blob
import os
data_folder = os.path.join(os.getcwd(), 'data_memap')
os.makedirs(data_folder, exist_ok=True)
data_folder
x_train_file_loc=data_folder+'/X_train.npy'

#create a memmap to store large numpy files
fp_x_train_memmap=np.memmap(x_train_file_loc,dtype=X_train.dtype,shape=X_train.shape,mode='w+')
fp_x_train_memmap[:]= X_train[:]
fp_x_train_memmap.shape

(50000, 256, 256, 1)

In [14]:
y_train_file_loc=data_folder+'/y_train.npy'
#create a memmap to store large numpy files
fp_y_train_memmap=np.memmap(y_train_file_loc,dtype=y_train.dtype,shape=y_train.shape,mode='w+')

In [15]:
fp_y_train_memmap[:]=y_train[:]
fp_y_train_memmap.shape

(50000, 256, 256, 1)

In [18]:
x_val_file_loc=data_folder+'/X_valid.npy'

#create a memmap to store large numpy files
fp_x_val_memmap=np.memmap(x_val_file_loc,dtype=X_val.dtype,shape=X_val.shape,mode='w+')
fp_x_val_memmap[:]= X_val[:]
fp_x_val_memmap.shape

y_val_file_loc=data_folder+'/y_valid.npy'

#create a memmap to store large numpy files
fp_y_val_memmap=np.memmap(y_val_file_loc,dtype=y_val.dtype,shape=y_val.shape,mode='w+')
fp_y_val_memmap[:]= y_val[:]
fp_y_val_memmap.shape

(10000, 256, 256, 1)

In [20]:
x_test_file_loc=data_folder+'/X_test.npy'

#create a memmap to store large numpy files
fp_x_test_memmap=np.memmap(x_test_file_loc,dtype=X_test.dtype,shape=X_test.shape,mode='w+')
fp_x_test_memmap[:]= X_test[:]
fp_x_val_memmap.shape

y_test_file_loc=data_folder+'/y_test.npy'

#create a memmap to store large numpy files
fp_y_test_memmap=np.memmap(y_test_file_loc,dtype=y_test.dtype,shape=y_test.shape,mode='w+')
fp_y_test_memmap[:]= y_test[:]
fp_y_test_memmap.shape

(20000, 256, 256, 1)

In [6]:
from azureml.core import Workspace, Dataset

#get workspace
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: cirrustest2
Azure region: southcentralus
Subscription id: c1d0a0ea-bf6e-4c1c-8b55-f1bdb0208df8
Resource group: awe-cirrus-rg


In [7]:
data_folder = os.path.join(os.getcwd(), 'data_memap')
print(data_folder)

/mnt/batch/tasks/shared/LS_root/mounts/clusters/sn-gpu-lab/code/users/supanaesw/hyperParameterTuningTests/hyperdrive/train-hyperparameter-tune-deploy-with-keras/gpu_experiments/data_memap


In [10]:
import os

# get the datastore to upload prepared data
datastore = ws.get_default_datastore()
# upload the local file from src_dir to the target_path in datastore
#datastore.upload(src_dir='data', target_path='data')
print(datastore)
#data_folder = os.path.join(os.getcwd(), 'data')
# os.makedirs(data_folder, exist_ok=True)
datastore_paths=[(datastore,data_folder)]
dataset = Dataset.File.from_files(path = datastore_paths)
print(dataset)

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-83a404c3-cbf3-4749-99c5-dfe0ca674e95",
  "account_name": "cirrustest28363900696",
  "protocol": "https",
  "endpoint": "core.windows.net"
}


DatasetValidationError: Cannot load any data from the specified path. Make sure the path is accessible and contains data.
(No files were found using path provided. Please make sure the path you've specified is correct, files exist and can be accessed.)|session_id=95e1ca3e-d5c0-4d02-8fb0-b16a27fd2748